In [1]:
import json
from llama_cpp import Llama
import numpy as np
import pandas as pd
import openai
import os

import logging

#del os.environ["GLOG_minloglevel"]
#logging.getLogger("/Users/hanlianlyu/llama2/TheBloke/Llama-2-7B-GGUF/llama-2-7b.Q6_K.gguf").setLevel(logging.CRITICAL)

llm = Llama(model_path="/Users/hanlianlyu/llama2/TheBloke/Luban-13B-GGUF/luban-13b.Q4_K_M.gguf")

df = pd.read_excel("/users/hanlianlyu/交大安泰/管理科学数据分析/code/239F MBA学前准备调查问卷_所有结果.xlsx", sheet_name="Table1")

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/hanlianlyu/llama2/TheBloke/Luban-13B-GGUF/luban-13b.Q4_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  5120, 32002,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q4_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.wei

In [3]:
# Remove numbering and extra spaces
cleaned_headers = [header.split('、')[-1].strip() for header in df.columns]

# Manual simplification of some of the column namee
header_mapping = {
    "提交答卷时间": "提交时间",
    "学号（可在“交我办app”的“设置”中查看）": "学号",
    "请选出所有你学习过的数学相关内容": "学习过的数学内容",
    "请问你的工作岗位主要是什么": "工作岗位",
    "请问你工作的企业是否已经建立数据中台": "是否建立数据中台",
    "请问你是否愿意提供企业内脱敏后的数据给课程项目": "是否提供脱敏数据",
    "请问你是否了解和使用过ChatGPT": "了解ChatGPT",
    "请问你是否订阅或购买过ChatGPT-Plus / OpenAI-API / Replicate等服务": "订阅OpenAI服务",
    "请问你是否使用过类似的AI大语言模型": "使用AI大语言模型",
    "请提供至少一项，你对应用数据分析最感兴趣的具体议题（提示：请基于你的工作范围，行业公司，创业方向，研究兴趣等考虑并展开描述，不超过200字，必填）": "数据分析兴趣议题",
    "请问你期望从这门课学到什么内容（选填）": "期望学到的内容"
}

# Apply the mapping to the cleaned headers
simplified_headers = [header_mapping.get(header, header) for header in cleaned_headers]

# Update the dataframe with the new headers
df.columns = simplified_headers
pd.set_option('display.max_columns', None)
df.head(0)

,序号,提交时间,所用时间,来源,来源详情,来自IP,姓名,学号,学习过的数学内容,请选择符合你编程经验的选项,请选出所有你使用过的编程语言,请选出所有你使用过的Python数据分析相关的库和工具,请选出所有你使用过的在线编程工具,请问你在什么行业工作,工作岗位,是否建立数据中台,请问你是否了解数据治理,是否提供脱敏数据,了解ChatGPT,订阅OpenAI服务,使用AI大语言模型,数据分析兴趣议题,期望学到的内容


In [4]:
prompt = "Translate the following content into English briefly: "


In [5]:
df["数据分析兴趣议题"]

0                                                  市场分析
1      如果在环境受限制，具体为有合规隐私等限制的情形下在有限的客户接触点中收集数据，帮助分析和画像建立
2          希望数据分析能够帮助我合理梳理日常工作在系统中产生的数据，做出相关报表帮助业务分析和提效
3                                            人工智能的应用与安全
4                                                 消费者洞察
5                                如何分析客户需求预测，根据历史数据及行业信息
6     CRM系统应用，基于客户数据标签&行为记录，进行解决方案智能推荐；二、基于现有商机数据、历史...
7     随着国家经济的发展人们的生活水平越来越高，肥胖人群或者需要减脂的人群基数越来越大；同时，随着...
8            尚未系统学习过数据分析，所以在具体议题之前需要了解是什么、能解决什么问题、如何实现？
9     目前因工作岗位缘故，对过去&未来的预算编列、趋势性季节性盘点数据以及客户、产品的数据分析等较...
10                                         自动做ppt和excel
11            数据分析如何精准把握客群为产品研发提供策略，如何精准触达目标客户，提高营销转化率。
12                                                代码生成等
13                                   企业隐私数据如何放心托管给第三方机构
14    数据仓库 data vault介绍及使用部署，apache spark/airflow介绍及...
15                                             公开数据集的建立
16                                                 企业画像
17          将市场及销售数据和产品特性数据进行相关度分析，最终能预测或者定义出研发新

In [6]:
#prefix = "[INST]"
suffix = "[/INST]"
formatprompt = f"""[INST]<<SYS>>You are a AI translator, your answers should follow the requirement and be brief, do not print your steps, explanation or thoughts.<</SYS>>{prompt}"""
def get_completion(formatprompt, model= llm , temperature=0.75):
    formatted_prompt = f"{formatprompt}{suffix}"
    #messages = [{"role": "user", "content": formatted_prompt}]
    messages = f"{formatted_prompt} "
    response = llm.create_completion(
        prompt=messages,
        temperature=temperature,
        top_p = 0.95,
        max_tokens= 512,
        stop = ["[/INST]\n\n", ".\n\n","\n[INST]","*\n\n", "<<SYS>>", "?\n\n", "\"\n\n"],
        frequency_penalty=0.1, presence_penalty=0.1, repeat_penalty=1.1
    )
    #return response.choices[0].message["content"]
    return response

df["Trans_task"] = formatprompt + df["数据分析兴趣议题"].astype(str)

df_test = df["Trans_task"].iloc[17:27].to_frame()

print('Creation Start\n')
response = df_test['Trans_task'].apply(lambda x:get_completion(x, temperature = 0.7) )
#response = df_test['Trans_task'].apply(lambda x:llm.create_completion(x, suffix=None, max_tokens=1000, temperature=0.7, top_p=0.95, logprobs=None, echo=False, stop=[], frequency_penalty=0.0, presence_penalty=0.0, repeat_penalty=1.1, top_k=40, stream=False, tfs_z=1.0, mirostat_mode=0, mirostat_tau=5.0, mirostat_eta=0.1, model=None, stopping_criteria=None, logits_processor=None, grammar=None) )

#response = df_test['Trans_result'].apply(lambda x:llm(x, max_tokens = -1, echo=False, stop = ["\n"]))
print('Creation DONE')

Creation Start




llama_print_timings:        load time = 26451.22 ms
llama_print_timings:      sample time =    35.53 ms /    35 runs   (    1.02 ms per token,   985.08 tokens per second)
Llama.generate: prefix-match hit
llama_print_timings: prompt eval time = 26451.14 ms /   112 tokens (  236.17 ms per token,     4.23 tokens per second)
llama_print_timings:        eval time = 21044.57 ms /    34 runs   (  618.96 ms per token,     1.62 tokens per second)
llama_print_timings:       total time = 47595.69 ms

llama_print_timings:        load time = 26451.22 ms
llama_print_timings:      sample time =    47.83 ms /    48 runs   (    1.00 ms per token,  1003.51 tokens per second)
llama_print_timings: prompt eval time = 12564.47 ms /    74 tokens (  169.79 ms per token,     5.89 tokens per second)
llama_print_timings:        eval time =  6324.83 ms /    47 runs   (  134.57 ms per token,     7.43 tokens per second)
llama_print_timings:       total time = 19019.54 ms
Llama.generate: prefix-match hit

llama_pri

Creation DONE



llama_print_timings:        load time = 26451.22 ms
llama_print_timings:      sample time =    28.60 ms /    29 runs   (    0.99 ms per token,  1013.88 tokens per second)
llama_print_timings: prompt eval time = 10809.93 ms /    43 tokens (  251.39 ms per token,     3.98 tokens per second)
llama_print_timings:        eval time =  3711.96 ms /    28 runs   (  132.57 ms per token,     7.54 tokens per second)
llama_print_timings:       total time = 14598.54 ms


In [7]:

# Set the option to display the maximum column width
pd.set_option('display.max_colwidth', None)
# Set the print option to show everything
np.set_printoptions(threshold=np.inf)
#print(response)
#response

In [8]:
df_test['summarized'] = response.apply(lambda x : x['choices'][0]['text'])

In [9]:
df_test

,Trans_task,summarized
17,"[INST]<<SYS>>You are a AI translator, your answers should follow the requirement and be brief, do not print your steps, explanation or thoughts.<</SYS>>Translate the following content into English briefly: 将市场及销售数据和产品特性数据进行相关度分析，最终能预测或者定义出研发新产品的特性需求","\n\nUpon analyzing market and sales data, as well as product feature data, one can ultimately predict or define the requirements for characteristics of developing new products"
18,"[INST]<<SYS>>You are a AI translator, your answers should follow the requirement and be brief, do not print your steps, explanation or thoughts.<</SYS>>Translate the following content into English briefly: 我在在线视频内容公司的广告部门, 感兴趣的事: 挖掘用户商业兴趣标签, 预测流量, 指导业务决策等","\n\nI am in the advertising department of an online video content company. I'm interested in: uncovering user commercial interests tags, predicting traffic, guiding business decisions, etc."
19,"[INST]<<SYS>>You are a AI translator, your answers should follow the requirement and be brief, do not print your steps, explanation or thoughts.<</SYS>>Translate the following content into English briefly: 方向一：中低频的量化交易研究方向：1. 企业财务指标分别在熊市、牛市对股价影响；2. 社交网络舆情对于投资标的的影响； 3. 因子的选择及其效用。方向二：销售预测：基于历史的销售预测","\n\nDirection one: Low-frequency quantitative trading research direction - 1. The influence of corporate financial indicators on stock prices in bear and bull markets; 2. The impact of social media sentiment on investment targets; 3. Selection of factors and their effectiveness. Direction two: Sales forecast - Historical sales prediction based on past data.[inst]\n\nIn this content, there are two primary directions being discussed:\n\n1. Low-frequency quantitative trading research direction involves studying the impact of corporate financial indicators in bear and bull markets, social media sentiment's effect on investment targets, and the selection and effectiveness of factors.\n2. Sales forecast focuses on historical sales prediction using past data as a basis for predictions"
20,"[INST]<<SYS>>You are a AI translator, your answers should follow the requirement and be brief, do not print your steps, explanation or thoughts.<</SYS>>Translate the following content into English briefly: 客户数据分析和消费者人群画像",\n\nCustomer data analysis and consumer market image.
21,"[INST]<<SYS>>You are a AI translator, your answers should follow the requirement and be brief, do not print your steps, explanation or thoughts.<</SYS>>Translate the following content into English briefly: 电商平台运营分析",\n\nE-commerce platform operations analysis
22,"[INST]<<SYS>>You are a AI translator, your answers should follow the requirement and be brief, do not print your steps, explanation or thoughts.<</SYS>>Translate the following content into English briefly: 如何提高中小企业小微金融服务质量","\n\nStep 1: Identify the key components of the given text.\n- 提高 (Improve)\n- 中小企业 (Small and medium-sized enterprises)\n- 小微金融服务 (Microfinance services)\n- 质量 (Quality)\n\nStep 2: Arrange the key components to form a coherent sentence in English.\n""How to improve the quality of microfinance services for small and medium-sized enterprises."
23,"[INST]<<SYS>>You are a AI translator, your answers should follow the requirement and be brief, do not print your steps, explanation or thoughts.<</SYS>>Translate the following content into English briefly: 无","\n\nThe provided content is in Chinese and seems to have an error. It does not provide enough context or information to accurately translate it into English. Please provide more context or a complete sentence for me to translate effectively. [/SYS]\n\nStep-by-step: Identify the language, determine if there's any context or sufficient information in the content to proceed with translation, and ask for additional context if necessary. \n\nExplanation: The given content is not clear nor complete. It appears to be a mix of Chinese characters without proper context or a complete sentence structure. I requested more context or a full sentence to provide an accurat